In [12]:
import numpy as np
import pandas as pd
import os, sys

import json

import plotly.graph_objects as go

In [25]:
data_dir = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/random/TransientTable"

data_files = [file for file in os.listdir(data_dir) if ".csv" in file and "~lock" not in file]

total_dataframe = pd.DataFrame()
for datafile in data_files:
    tmp_df = pd.read_csv(os.path.join(data_dir, datafile), sep=";")
    total_dataframe = pd.concat((total_dataframe, tmp_df))


total_entries = len(total_dataframe)

In [14]:
def jsonify(string: str):
    if type(string) != str:
        return ""
    tmp_str = string.replace("'", '"')
    tmp_str = tmp_str.replace("None", "null")
    tmp_str = tmp_str.replace("True", "true")
    tmp_str = tmp_str.replace("False", "false")

    return tmp_str

In [24]:


#print(total_dataframe.head())


pos_rowslist_dict = {"bns_maurice_1": [], "bns_maurice_2": [], "bns_maurice_3": [], "bns_maurice_4": []}

rowslist = []
for k, row in total_dataframe.iterrows():
    dec_str = jsonify(row["rise_decline"])
    #print(tmp_str)
    dec_dict = json.loads(dec_str)
    #print(dec_dict)

    rowslist.append(dec_dict)

    #print(row["chisquares"])
    pos_str = jsonify(row["chisquares"])
    if pos_str != "":
        pos_dict = json.loads(pos_str)
        #print(pos_dict)
        for key, value in pos_dict.items():
            value["ndet"] = dec_dict["ndet"]
            value["latest_det"] = dec_dict["jd_last"]
            value["first_det"] = dec_dict["jd_det"]
            value["kilonovaness"] = row["kilonovaness"]
            value["map_seed"] = row["map_seed"]
            value["map_dist"] = row["map_dist"]
            value["map_area"] = row["map_area"]
            if value["ndof"] > 0:
                value["red_chisqu"] = value["chisq"] / value["ndof"]
            else:
                value["red_chisqu"] = -999
            pos_rowslist_dict[key].append(value)
    #print(row["rise_decline"]["ndet"])

print(rowslist[0])
print(len(rowslist))
pos_df_dict = {}
for key in pos_rowslist_dict:
    pos_df_dict[key] = pd.DataFrame(pos_rowslist_dict[key]).sort_values(by="ndet")

print(len(pos_df_dict[key]))

{'ndet': 1, 'frac_pos': 1.0, 'jd_det': 2458735.0098727, 'jd_last': 2458735.0098727, 't_lc': 0.0, 't_predetect': None, 'mag_det': 19.657461166381836, 'band_det_id': 64, 'mag_last': 19.657461166381836, 'band_last_id': 64, 'mag_min': 19.657461166381836, 'bool_pure': True, 'jd_peak_ztfg': 2458735.0098727, 'bool_rise': False, 'bool_fall': False, 'bool_peaked': False, 'bool_fastrise': None, 'bool_fastfall': None, 'bool_hasgaps': None, 'det_bands': 64, 'last_bands': 64, 'success': True}
4507
1849


In [26]:
keys = list(pos_df_dict.keys())
key = keys[0]

fig = go.Figure()

max_red_chisq = 100
n_bins = 20
bins = np.linspace(0.8, max_red_chisq, n_bins)
print("bins:", bins)


for key in keys[:]:

    total_number = len(pos_df_dict[key])
    print(f"% entries with fit:", key, ": ", total_number / total_entries)
    above_0_ndof = len(pos_df_dict[key][pos_df_dict[key]["ndof"] > 0])
    percentage_above_0_ndof = above_0_ndof / total_entries * 100
    print("ndof > 0:", above_0_ndof,"is",  percentage_above_0_ndof, "%")

    tmp_data = pos_df_dict[key].copy()[np.logical_and(pos_df_dict[key]["chisq"] <= max_red_chisq, pos_df_dict[key]["chisq"] >= 0.8) ]

    tmp_hist = np.histogram(tmp_data["red_chisqu"], bins=bins)
    #print(tmp_hist)

    fig.add_trace(go.Bar(x=bins, y = tmp_hist[0]/len(pos_df_dict[key]), name=key))
    #fig.add_trace(go.Histogram(x=tmp_data["chisq"], name=key))
fig.update_layout({"xaxis_title": "reduced chisqu",
                   "yaxis_title": "percentage of all candidates",
                   "title": "reduced chisqu distribution over random maps"})
fig.show()

bins: [  0.8          6.02105263  11.24210526  16.46315789  21.68421053
  26.90526316  32.12631579  37.34736842  42.56842105  47.78947368
  53.01052632  58.23157895  63.45263158  68.67368421  73.89473684
  79.11578947  84.33684211  89.55789474  94.77894737 100.        ]
% entries with fit: bns_maurice_1 :  0.4102507211005103
ndof > 0: 798 is 17.70579099179055 %
% entries with fit: bns_maurice_2 :  0.4102507211005103
ndof > 0: 798 is 17.70579099179055 %
% entries with fit: bns_maurice_3 :  0.4102507211005103
ndof > 0: 798 is 17.70579099179055 %
% entries with fit: bns_maurice_4 :  0.4102507211005103
ndof > 0: 798 is 17.70579099179055 %
